In [1]:
import requests
import time
import re
import base64

# ================= CONFIGURAZIONE =================
API_KEY = "c74e4f84aa1b66630caabecaf7d78fd21b546e2f70bb385f1c1118f12b72443e"
# ==================================================

# La tua lista di IoC (Ho rimesso quelli del tuo output precedente e altri)
IOC_LIST = [
    '185.146.173.58', 'AB4F368345DC20DC59516C098AAD331C',
    'https://happytiger.co.uk', '51.158.150.210', '94.228.165.234',
    '49.13.69.46', '754FA0CFF33F2F59F4B8CA3DEB4C2CDA', '1B0D4EE4D3E20B8922251941B956C21E',
    '11eee4d6646c2f90a50889c8848a6b69', '7a136ea826815ac22f2849e099a1c6a1',
    'https://w0rm.pw', 'http://213.249.19.49:8080', '213.249.19.49',
    '185.146.173.59', '4612AD25981D7183127316392178B142', '193.29.139.248',
    '104.18.154.42', '160.153.0.179', 'D3819AB2A174B1DEFD45DDEB5E5495FF',
    '217.196.103.206', '109.107.172.30', '31.214.168.228', 'D1B822529EB33B316FF2C926DFF30D12'
]

HEADERS = {"x-apikey": API_KEY}

def get_ioc_type(ioc):
    if re.match(r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$", ioc):
        return "ip_addresses"
    elif re.match(r"^[a-fA-F0-9]{32}$", ioc) or re.match(r"^[a-fA-F0-9]{64}$", ioc):
        return "files"
    else:
        return "urls"

def check_virustotal_detailed(ioc):
    ioc_type = get_ioc_type(ioc)
    
    # URL Handling
    if ioc_type == "urls":
        url_id = base64.urlsafe_b64encode(ioc.encode()).decode().strip("=")
        url = f"https://www.virustotal.com/api/v3/urls/{url_id}"
    else:
        url = f"https://www.virustotal.com/api/v3/{ioc_type}/{ioc}"

    try:
        response = requests.get(url, headers=HEADERS)
        
        if response.status_code == 200:
            data = response.json()
            attrs = data['data']['attributes'] # Prendiamo tutti gli attributi
            stats = attrs['last_analysis_stats']
            
            malicious = stats['malicious']
            
            # OUTPUT BASE
            if malicious > 0:
                print(f"[{ioc_type[:-1].upper()}] {ioc} -> \033[91mMALICIOUS ({malicious}/{sum(stats.values())})\033[0m")
                
                # --- EXTRACTION DETAILS (SOLO SE MALICIOUS) ---
                print("    ℹ️  DETAILS:")
                
                # 1. TAGS (Comune a tutti)
                tags = attrs.get('tags', [])
                if tags: print(f"       🏷️  Tags: {', '.join(tags)}")
                
                # 2. DETTAGLI PER IP
                if ioc_type == "ip_addresses":
                    asn = attrs.get('as_owner', 'N/A')
                    country = attrs.get('country', 'N/A')
                    print(f"       🌍  Owner/ASN: {asn}")
                    print(f"       🏳️  Country: {country}")
                
                # 3. DETTAGLI PER URL
                elif ioc_type == "urls":
                    title = attrs.get('title', 'N/A')
                    categories = attrs.get('categories', {})
                    # Prendiamo solo i valori delle categorie (es. Fortinet: Phishing)
                    cat_list = list(set(categories.values())) 
                    
                    print(f"       📑  Page Title: {title}")
                    if cat_list: print(f"       🗂️  Categories: {', '.join(cat_list)}")
                
                # 4. DETTAGLI PER FILE
                elif ioc_type == "files":
                    names = attrs.get('names', [])
                    type_desc = attrs.get('type_description', 'N/A')
                    print(f"       📄  Type: {type_desc}")
                    if names: print(f"       📛  Known Names: {names[:3]}") # Primi 3 nomi
                
                print("") # Riga vuota per separare
                
            else:
                print(f"[{ioc_type[:-1].upper()}] {ioc} -> \033[92mCLEAN\033[0m")
            
        elif response.status_code == 404:
            print(f"[{ioc_type[:-1].upper()}] {ioc} -> Not Found")
        else:
            print(f"[{ioc_type[:-1].upper()}] {ioc} -> Error {response.status_code}")
            
    except Exception as e:
        print(f"Error checking {ioc}: {e}")

print(f"Starting detailed check for {len(IOC_LIST)} IoCs...")
print("Rate limit: 1 request every 20 seconds (Strict Safety)\n")

for i, ioc in enumerate(IOC_LIST):
    # Stampa di progresso sulla stessa riga
    print(f"Checking {i+1}/{len(IOC_LIST)}...", end=" ", flush=True)
    
    # Esegue il check e va a capo automaticamente
    check_virustotal_detailed(ioc)
    
    if i < len(IOC_LIST) - 1:
        time.sleep(20) # 20 secondi per stare sicuri (limite è 15s)

print("\nDone!")

Starting detailed check for 23 IoCs...
Rate limit: 1 request every 20 seconds (Strict Safety)

Checking 1/23... [IP_ADDRESSE] 185.146.173.58 -> MALICIOUS (1/95)
    ℹ️  DETAILS:
       🌍  Owner/ASN: CLOUDFLARENET
       🏳️  Country: SE

Checking 2/23... [FILE] AB4F368345DC20DC59516C098AAD331C -> Not Found
Checking 3/23... [URL] https://happytiger.co.uk -> MALICIOUS (2/98)
    ℹ️  DETAILS:
       🏷️  Tags: external-resources
       📑  Page Title: Just a moment...
       🗂️  Categories: Gambling (alphaMountain.ai), shopping, gambling

Checking 4/23... [IP_ADDRESSE] 51.158.150.210 -> MALICIOUS (1/95)
    ℹ️  DETAILS:
       🌍  Owner/ASN: Scaleway S.a.s.
       🏳️  Country: NL

Checking 5/23... [IP_ADDRESSE] 94.228.165.234 -> CLEAN
Checking 6/23... [IP_ADDRESSE] 49.13.69.46 -> CLEAN
Checking 7/23... [FILE] 754FA0CFF33F2F59F4B8CA3DEB4C2CDA -> Not Found
Checking 8/23... [FILE] 1B0D4EE4D3E20B8922251941B956C21E -> Not Found
Checking 9/23... [FILE] 11eee4d6646c2f90a50889c8848a6b69 -> Not Found
